In [ ]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_mcp_adapters.client import MultiServerMCPClient

# 1. Setup API Keys
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# 2. Connect to the MCP Server
mcp_client = MultiServerMCPClient({
    "trading_server": {
        "transport": "stdio",
        "command": "python",
        "args": ["mcp_server.py"]
    }
})

# 3. Fetch the actual tool objects from the server
# This is the missing step in your previous version!
all_mcp_tools = await mcp_client.get_tools()
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

print(f" Loaded {len(all_mcp_tools)} tools from MCP Server.")

In [ ]:
# Trend Agent setup
trend_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Trend Analyst. Use the provided tools to calculate drift (mu)."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Use all_mcp_tools instead of undefined variables
trend_agent = create_openai_functions_agent(model, all_mcp_tools, trend_prompt)
trend_executor = AgentExecutor(agent=trend_agent, tools=all_mcp_tools, verbose=True)

# Noise Agent setup
noise_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Sentiment Analyst. Search for news that might affect stock noise."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

noise_agent = create_openai_functions_agent(model, all_mcp_tools, noise_prompt)
noise_executor = AgentExecutor(agent=noise_agent, tools=all_mcp_tools, verbose=True)

In [ ]:
from langchain.tools import tool

@tool 
def trend_agent_tool(ticker: str):
    """Calls the Trend Agent to get the mu (drift) value."""
    return trend_executor.invoke({"input": f"Analyze the trend for {ticker}"})

# Create a master tool list for the supervisor
sup_tools = all_mcp_tools + [trend_agent_tool]

sup_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are the Portfolio Supervisor. 
    1. Call market_data_tool to get prices.
    2. Use the trend_agent_tool to get mu.
    3. Call mpc_execution_tool using that mu and those prices.
    4. Provide a final recommendation."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

sup_agent = create_openai_functions_agent(model, sup_tools, sup_prompt)
sup_executor = AgentExecutor(agent=sup_agent, tools=sup_tools, verbose=True)

# Test it!
await sup_executor.ainvoke({"input": "Should I buy AAPL?"})